<h2>Entity Resolution

<ul>
    <li>This code snippet sets up a PySpark environment in a Python script.</li><li> It first imports the necessary modules like pyspark, os, and sys.</li><li> Then, it sets the Python executable for PySpark to the same one being used by the script.</li><li> Finally, it imports the SparkContext class for creating RDDs and the SparkSession class for programming Spark with the DataFrame API.</li>
    </ul>

In [ ]:
import pyspark
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession

<ul><li>This code creates a SparkSession named spark with specific configuration options. It sets the driver memory to 16 GB and names the application 'chapter_2'.</li><ul>

In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_2').getOrCreate()

<h3>Setting up our data</h3>

<ul><li>This shell script creates a directory named "linkage," navigates into it, downloads a file named "donation.zip" using curl, and then unzips it. Finally, it unzips all files matching the pattern "block_*.zip" within the "linkage" directory.</li></ul>

<ul><li>This code reads a CSV file named "block_1.csv" located in the "data/linkage/donation/block_1" directory into a Spark DataFrame called prev. It then displays the first two rows of the DataFrame using show(2) and subsequently displays the entire DataFrame prev.</li></ul>

In [ ]:
prev = spark.read.csv("data/linkage/donation/block_1/block_1.csv")
prev.show(2)
prev

<ul><li>This code reads a CSV file named "block_1.csv" into a Spark DataFrame called parsed. Further displaying its schema.</ul></li>

In [ ]:
parsed =spark.read.option("header","true").option("nullValue", "?").\option("inferSchema", "true").csv("data/linkage/donation/block_1/block_1.csv")

<ul><li>This code first calculates the number of rows in the parsed DataFrame. It then caches the parsed DataFrame in memory for faster access in subsequent operations.</li></ul>

In [ ]:
parsed.printSchema()
parsed.show(5)
parsed.count()
parsed.cache()

<ul><li>This code snippet groups the parsed DataFrame by the "is_match" column and calculates the count of each group. It then orders the result in descending order based on the count of each group and displays the result, showing the number of occurrences for each "is_match" value, with the most frequent ones listed first.</ul></li>

In [ ]:
from pyspark.sql.functions import col
parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

<ul><li>This code creates a temporary view called "linkage" from the parsed DataFrame. This temporary view allows you to query the DataFrame using SQL syntax.</li></ul>

In [ ]:
parsed.createOrReplaceTempView("linkage")

<ul><li>This Spark SQL query selects the "is_match" column and counts the number of occurrences for each value. It groups the results by "is_match" and orders them in descending order based on the count. Finally, it displays the "is_match" values along with their respective counts.</li></ul>

In [ ]:
spark.sql("""
SELECT is_match, COUNT(*) cnt
FROM linkage
GROUP BY is_match
ORDER BY cnt DESC
""").show()

<h3>Fast Summary Statistics for DataFrames</h3>

<b>This code computes summary statistics for specific columns in the parsed DataFrame.
</b>
<ul><li>summary = parsed.describe() calculates summary statistics for all columns in parsed and selects and displays summary statistics for the "cmp_fname_c1" and "cmp_fname_c2" columns from summary.</li></ul>
<b>
It then separates the data into two DataFrames based on the "is_match" column.</b>
<ul><li>
matches = parsed.where("is_match = true") filters parsed to include only rows where "is_match" is true.
    </li><li>
match_summary = matches.describe() computes summary statistics for matches.
    </li><li>
misses = parsed.filter(col("is_match") == False) filters parsed to include only rows where "is_match" is false.
    </li><li>
miss_summary = misses.describe() computes summary statistics for misses.
    </li></ul>







In [ ]:
summary = parsed.describe()
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

matches = parsed.where("is_match = true")
match_summary = matches.describe()
misses = parsed.filter(col("is_match") == False)
miss_summary = misses.describe()

<h3>Pivoting and Reshaping Dataframes</h3>

<ul><li>This code converts summary statistics from a PySpark DataFrame to a Pandas DataFrame for easier analysis in Python. It reshapes the Pandas DataFrame by transposing it and resetting the index, making it easier to work with. After renaming columns and removing the axis name, the Pandas DataFrame is converted back to a Spark DataFrame for further processing.</li></ul>

In [ ]:
summary_p = summary.toPandas()
summary_p.head()
summary_p.shape
summary_p = summary_p.set_index('summary').transpose().reset_index()
summary_p = summary_p.rename(columns={'index':'field'})
summary_p = summary_p.rename_axis(None, axis=1)
summary_p.shape
summaryT = spark.createDataFrame(summary_p)
summaryT.printSchema()

<ul><li>This function pivot_summary(desc) converts summary statistics from a PySpark DataFrame desc to a more manageable format by first converting it to a Pandas DataFrame.</li><li> It then transposes the Pandas DataFrame, sets the 'summary' column as the index, and resets the index.</li><li> After renaming columns and removing the axis name, the Pandas DataFrame is converted back to a Spark DataFrame descT.</li><li> Finally, the function converts the metric columns to DoubleType from string format before returning the modified Spark DataFrame descT.</li></ul>

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType
def pivot_summary(desc):
    desc_p = desc.toPandas()
    desc_p = desc_p.set_index('summary').transpose().reset_index()
    desc_p = desc_p.rename(columns={'index':'field'})
    desc_p = desc_p.rename_axis(None, axis=1)
    descT = spark.createDataFrame(desc_p)
    for c in descT.columns:
        if c == 'field':
            continue
        else:
            descT = descT.withColumn(c, descT[c].cast(DoubleType()))
        return descT

<ul><li>It uses a function pivot_summary that converts each DataFrame to a Pandas DataFrame, transposes it, resets the index, renames columns, and then converts it back to a Spark DataFrame (match_summaryT and miss_summaryT). </li></ul>

In [ ]:
match_summaryT = pivot_summary(match_summary)
miss_summaryT = pivot_summary(miss_summary

<h3>Joining DataFrames and Selecting Features</h3>

<ul><li>This code creates temporary views for the match_summaryT DataFrame as "match_desc" and the miss_summaryT DataFrame as "miss_desc". </li><li>It then performs a SQL query that joins these views on the "field" column, calculates the total count for each field by adding the counts from the match and miss DataFrames, and computes the difference in means (delta) between the match and miss DataFrames for each field.</li><li> It filters out fields "id_1" and "id_2" and orders the results by delta in descending order, followed by total in descending order.</li></ul>

In [ ]:
match_summaryT.createOrReplaceTempView("match_desc")
miss_summaryT.createOrReplaceTempView("miss_desc")
spark.sql("""
SELECT a.field, a.count + b.count total, a.mean- b.mean delta
FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
WHERE a.field NOT IN ("id_1", "id_2")
ORDER BY delta DESC, total DESC
""")

<h3>Scoring and Model Evaluation</h3>

<ul><li>This code snippet defines a list of good features named good_features that are selected for further analysis.</li><li> It then creates a sum expression by joining these features with the "+" operator, which can be used in mathematical operations.</li></ul>

In [ ]:
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]
sum_expression = " + ".join(good_features)
sum_expression

<ul><li>This code snippet uses the good_features list to fill null values in the parsed DataFrame with 0 for the selected features.</li><li> It then calculates a new column 'score' by summing up the values of the selected features using the expr function with the sum_expression. </li><li>Finally, it selects the 'score' and 'is_match' columns from the DataFrame and displays the result using the show method.</li></ul>

In [ ]:
from pyspark.sql.functions import expr
scored = parsed.fillna(0, subset=good_features).\
withColumn('score', expr(sum_expression)).\
select('score', 'is_match')
scored.show()

<ul><li>This function computes a cross-tabulation between two columns, 'above' (indicating if a score is above a threshold) and 'is_match'. It groups the data by 'above' and 'is_match' values, then counts the occurrences of 'true' and 'false' in 'is_match' for each 'above' value.</li></ul>

In [ ]:
def crossTabs(scored: DataFrame, t: DoubleType)-> DataFrame:
    return scored.selectExpr(f"score >= {t} as above", "is_match").\groupBy("above").pivot("is_match", ("true", "false")).\count()

<ul><li>It computes a cross-tabulation between the 'above' column (indicating if the score is above or equal to 4.0) and the 'is_match' column. </li></ul>

In [ ]:
crossTabs(scored, 4.0).show()